<a href="https://colab.research.google.com/github/savaken077/Asobi3/blob/main/20220422_%E9%AB%98%E8%A7%A3%E5%83%8F%E5%BA%A6%E9%A2%A8%E4%B8%8A%E6%B3%95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
jmax = 101
dt = 0.02

In [6]:
gamma = 1.4

In [7]:
PI  = 1.0
RHOI = 1.0
UI = 0.0

PE = 0.1
RHOE = 0.1
UE = 0.0

In [8]:
xmin, xmid, xmax = 0.0, 0.5, 1.0
x = np.linspace(xmin, xmax, jmax)
dx = (xmax - xmin) / (jmax - 1)
dtdx = dt / dx

Roeスキームによる計算

In [9]:
def init():
  Q = np.zeros([jmax, 3])
  Q[x <= xmid, 0] = RHOI
  Q[x <= xmid, 1] = RHOI = UI
  Q[x <= xmid, 2] = (PI/ (gamma -1.0) + 0.5 * RHOI *UI **2)

  Q[x > xmid, 0] = RHOE
  Q[x > xmid, 1] = RHOE * UE
  Q[x > xmid, 2] = (PE / (gamma - 1.0) + 0.5 * PHOE * UE **2)

  return Q

In [10]:
def calc_CFL(Q):
  rho, rhou, e = Q[:,0], Q[:,1], Q[:,2]

  u = rhou / rho
  p = (gamma - 1.0) * (e - 0.5 * rho * u **2)

  c = np.sqrt(gamma * p / rho)
  sp = c + np.abs(u)
  return max(sp) * dtdx

In [11]:
def Roe_flux(QL, QR, E):
  for j in range(jmax - 1):
    rhoL, uL, pL = QL[    j,0], QL[    j,1], QL[    j,2]
    rhoR, uR, pR = QR[j + 1,0], QR[j + 1,1], QR[j + 1,2]

    rhouL = rhoL * uL
    rhouR = rhoR * uR

    eL = pL / (gamma - 1.0) + 0.5 * rhoL * uL ** 2
    eR = pR / (gamma - 1.0) + 0.5 * rhoR * uR ** 2

    HL = (eL + pL) / rhoL
    HR = (eR + pR) / rhoR

    cL = np.sqrt((gamma - 1.0) * (HL - 0.5 * uL ** 2))
    cR = np.sqrt((gamma - 1.0) * (HR - 0.5 * uR ** 2))

    #Roe平均 式(6.36)
    sqrhoL = np.sqrt(rhoL)
    sqrhoR = np.sqrt(rhoR)

    rhoAVE = sqrhoL * sqrhoR
    uAVE = (sqrhoL * uL + sqrhoR * uR) / (sqrhoL + sqrhoR)
    HAVE = (sqrhoL * HL + sqrhoR * HR) / (sqrhoL + sqrhoR)
    cAVE = np.sqrt((gamma - 1.0)* (HAVE - 0.5 * uAVE ** 2))
    eAVE = rhoAVE * (HAVE - cAVE **2 / gamma)

    dQ = np.array([rhoR - rhoL, rhoR * uR - rhoL * uL, eR - eL])

    Lambda = np.diag([np.abs(uAVE - cAVE),
                                          np.abs(uAVE),
                                                      np.abs(uAVE + cAVE)])
    
    b1 = 0.5 *(gamma - 1.0) * uAVE ** 2 / cAVE ** 2
    b2 = (gamma - 1.0) /cAVE ** 2

    R = np.array([[1.0, 1.0, 1.0],
                  [uAVE - cAVE, uAVE, uAVE + cAVE],
                  [HAVE - uAVE * cAVE, 0.5 * uAVE ** 2, HAVE + uAVE * cAVE]])
    
    Rinv = np.array([[0.5 * (b1 + uAVE / cAVE), -0.5 * (b2 * uAVE + cAVE), 0.5 * b2],
                     [1.0 - b1, b2 * uAVE, -b2],
                     [0.5 * (b1 - uAVE / cAVE), -0.5 * (b2 * uAVE - cAVE), 0.5 * b2]])
    
    AQ = R @ Lambda @ Rinv @ dQ

    EL = np.array([rhoL * uL, pL + rhouL * uL, (eL + pL ) * uL])
    ER = np.array([rhoR * uR, pR + rhouR* uR, (eR + pR ) * uR])

    E[j] = 0.5 * (ER + EL - AQ) #式(6.43)

In [12]:
def minmod(x, y):
  sgn = np.sign(x)
  return sgn * np.maximum(np.minimum(np.abs(x), sgn * y), 0.0)
  

In [13]:
def MUSCL(Q, order, kappa ):
  rho, rhou, e = Q[: 0], Q[:, 1], Q[:, 2]
  Q[:, 1] = rhou / rho #u
  Q[:, 2] = (gamma - 1.0 ) * (e - 0.5 * rho * Q[:,1] **2) #p
  
  if order == 2 or order == 3:
    # 2nd / 3rd & minmod limitter
    dQ = np.zeros([jmax, 3])
    for j in range (1, jmax - 1):
      Dp[j] = minmod(dQ[j] , b * dQ[j - 1])#式(2.73a)
      Dm[j] = minmod(dQ[j - 1] , b * dQ[j])#式(2.73b)
    Dp[0] = Dp[1]
    Dm[0] = Dm[1]

    QL = Q.copy()
    QR = Q.copy()

    for j in range (1, jmax - 1):
      QL[j] += 0.25 * ((1.0 - kappa) + Dp[j] + (1.0 + kappa)) * Dm[j]
      QL[j] -= 0.25 * ((1.0 + kappa) + Dp[j] + (1.0 - kappa)) * Dm[j]
    
    else:
      #1st order
      QL = Q.copy
      QR = Q.copy
    
    return QL , QR
    

  

In [14]:
def Roe_FDS(Q, order, kappa, nmax, print_interval = 2):
  E = np.zeros([jmax, 3])
  for n in range(nmax):
    if n % print_interval == 0:
      print(f'n = {n : 4d} : CFL = {calc_CFL(Q) : .4f}')
    
    Qold = Q.copy
    coefs = [0.5, 1.0]
    for coef in coefs:
      QL, QR = MUSCL(Qold, order, kappa)

      Roe_flux(QL,QR, E)
      for j in range(1, jmax - 1):
        Qold[j] = Q[j] - coef * dtdx *(E[j] - E[j-1])
        Qold[0] = Q[0]
        Qold[-1]= Q[-1]

      Q[:] = Qold[:]

In [15]:
nmax = 100
print_interval = 4

order = 2
#-1 = 2nd order fully upwind
# 0  = 2nd order upwind bisaed
# 1/3  = 3rd order upwind bisaed
kappa = 0

Q = init()
Roe_FDS(Q, order, kappa, nmax, print_interval)

UnboundLocalError: ignored